In [46]:
#!pip install grequests
!pip install openpyxl  


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [47]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning, module="grequests")
os.environ['GEVENT_SUPPORT'] = 'True'
#import grequests
import requests
from bs4 import BeautifulSoup
import json
import concurrent.futures
import urllib3
import time

In [48]:
df = pd.read_excel("../data/labeled_dataset.xlsx").drop(["Unnamed: 0"], axis = 1)

c:\Users\Heliomar\Desktop\clustering\lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [49]:
df.columns

Index(['sentence', 'news_link', 'outlet', 'topic', 'type', 'group_id',
       'num_sent', 'Label_bias', 'Label_opinion', 'article', 'biased_words4'],
      dtype='object')

In [50]:
df.shape

(1700, 11)

In [51]:
df['type'].unique()

array(['center', 'left', 'right'], dtype=object)

In [52]:
df['topic'].unique()

array(['elections-2020', 'sport', 'immigration', 'environment',
       'abortion', 'student-debt', 'vaccines', 'white-nationalism',
       'coronavirus', 'trump-presidency', 'middle-class', 'gender',
       'international-politics-and-world-news', 'gun-control'],
      dtype=object)

In [54]:
import time
import urllib3
from bs4 import BeautifulSoup

def get_news(index, url):
    try:
        http = urllib3.PoolManager()
        response = http.request("GET", url)
    except Exception as e:
        return (index, ("ERRO - 1: " + str(e)))
    
    if response.status != 200:
        # Trata a resposta de erro
        if response.status == 429:
            time.sleep(3)
            return get_news(index, url)
        
        return (index, (f"ERRO - request: {response.status}"))
    
    soup = BeautifulSoup(response.data, "html.parser")
    paragrafos = soup.find_all("p")
    
    texto_paragrafos = []
    
    try:
        for paragrafo in paragrafos:
            texto_paragrafos.append(paragrafo.get_text())
    except Exception as e:
        return (index, ("ERRO - 2: " + str(e)))
    
    if len(texto_paragrafos) == 0:
        soup = BeautifulSoup(response.data, "html.parser")
        paragrafos = soup.find_all("p")
        
        try:
            for paragrafo in paragrafos:
                texto_paragrafos.append(paragrafo.get_text())
        except Exception as e:
            return (index, ("ERRO - 3: " + str(e)))
    
    return (index, tuple(texto_paragrafos))



# def get_news_batch(urls):
#     results = []

#     with concurrent.futures.ThreadPoolExecutor() as executor:
#         futures = [executor.submit(get_news, i, url) for i, url in enumerate(urls)]

#         for future in concurrent.futures.as_completed(futures):
#             result = future.result()
#             results.append(result)

#     return dict(results)


def get_news_batch(urls):
    results = []

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = []
        
        for i, url in enumerate(urls):
            if pd.isnull(df.at[i, 'article']):
                future = executor.submit(get_news, i, url)
                futures.append(future)
        
        for future in concurrent.futures.as_completed(futures):
            result = future.result()
            results.append(result)

    return dict(results)


# Crie uma lista de URLs a partir da coluna "link"
urls = df['news_link'].tolist()

# Verifique se a coluna "article" está vazia e a coluna "link" não está vazia
empty_article_mask = df['article'].isnull()
empty_link_mask = df['news_link'].isnull()
rows_to_delete = df[empty_article_mask & empty_link_mask].index

# Exclua as linhas correspondentes
df = df.drop(rows_to_delete)

# Obtenha os textos das notícias para preencher a coluna "article"
articles_dict = get_news_batch(urls)

# Preencha a coluna "article" com os textos obtidos
for i, article_text in articles_dict.items():
    df.at[i, 'article'] = article_text

In [55]:
df

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1.0,1.0,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']"
1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1.0,1.0,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",['bitter']
2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1.0,1.0,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis']
3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,1.0,1.0,Non-biased,No agreement,A professor who teaches climate change classes...,['legitimate']
4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1.0,1.0,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '..."
...,...,...,...,...,...,...,...,...,...,...,...
1695,In every case legislators are being swarmed by...,https://www.alternet.org/2019/04/republicans-w...,alternet,gender,left,85.0,1.0,Biased,Expresses writer’s opinion,There are many reasons that Republicans and co...,"['deceit', 'hysteria', 'swarmed', 'right-wing']"
1696,Polls show the transgender ideology is deeply ...,https://www.breitbart.com/politics/2019/02/21/...,breitbart,gender,right,85.0,1.0,Biased,Somewhat factual but also opinionated,A man’s penis becomes a female penis once a ma...,"['ideology', 'unpopular']"
1697,Democrats and Republicans stood and applauded ...,https://eu.usatoday.com/story/news/politics/20...,usa-today,gender,center,85.0,1.0,Non-biased,Entirely factual,(WASHINGTON – New Mexico Rep. Debra Haaland go...,['saluted']
1698,"As a self-described Democratic socialist, Sen....",https://www.foxnews.com/politics/how-did-berni...,fox-news,middle-class,right,85.0,1.0,Non-biased,Entirely factual,"As a self-described Democratic socialist, Sen....",['outspoken']


Problematicos:

In [ ]:
#zerados = df.loc[df['full_article'].apply(len) == 0]


In [57]:
errados_1 = df.loc[df['article'].apply(lambda x: 'ERRO - 1' in str(x))]

In [58]:
errados_1.shape

(7, 11)

In [59]:
errados_1

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
937,A former North Korean diplomat who staged a hi...,http://feeds.reuters.com/~r/Reuters/worldNews/...,reuters,international-politics-and-world-news,center,47.0,1.0,Non-biased,Entirely factual,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,[]
1017,In the past Putin has warmly welcomed home Rus...,http://feeds.reuters.com/~r/Reuters/worldNews/...,reuters,international-politics-and-world-news,center,51.0,1.0,Biased,No agreement,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,['warmly']
1073,Russia killed civilians in air strikes in Syri...,http://feeds.reuters.com/~r/Reuters/worldNews/...,reuters,international-politics-and-world-news,center,54.0,1.0,Non-biased,Entirely factual,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,[]
1238,Other European countries have also said they w...,http://feeds.reuters.com/~r/reuters/topNews/~3...,reuters,international-politics-and-world-news,center,62.0,1.0,Non-biased,Entirely factual,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,[]
1502,European states are trying to accelerate a pla...,http://feeds.reuters.com/~r/reuters/topNews/~3...,reuters,international-politics-and-world-news,center,76.0,1.0,Non-biased,Entirely factual,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,[]
1535,"If confirmed to have fled, Jo Song Gil would b...",http://feeds.reuters.com/~r/Reuters/worldNews/...,reuters,international-politics-and-world-news,center,77.0,1.0,Biased,Somewhat factual but also opinionated,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,"['oppressive', 'impoverished']"
1585,"Paul Whelan, the former U.S. Marine held in Mo...",http://feeds.reuters.com/~r/Reuters/worldNews/...,reuters,international-politics-and-world-news,center,80.0,1.0,Non-biased,Entirely factual,ERRO - 1: HTTPConnectionPool(host='feeds.reute...,[]


In [60]:
errados_2 = df.loc[df['article'].apply(lambda x: 'ERRO - 2' in str(x))]

In [61]:
errados_2.shape

(0, 11)

In [62]:
errados_request = df.loc[df['article'].apply(lambda x: 'ERRO - request' in str(x))]

In [64]:
errados_request.shape

(3, 11)

In [66]:
errados_request[['news_link', 'article']]

,news_link,article
160,https://www.msn.com/en-gb/news/world/procter-a...,ERRO - request: 410
913,https://www.msn.com/en-us/news/other/families-...,ERRO - request: 404
1319,https://www.msn.com/en-us/news/other/families-...,ERRO - request: 404


In [68]:
df_exploded = errados_request.explode("article")
valores_unicos = df_exploded["article"].unique()
del df_exploded
valores_unicos

array(['ERRO - request: 410', 'ERRO - request: 404'], dtype=object)

In [69]:
merged = df.merge(errados_request, how='left', indicator=True)
filtered_df = merged[merged['_merge'] == 'left_only'].drop(columns='_merge')

filtered_df = filtered_df.merge(errados_1, how= 'left', indicator = True)
filtered_df = filtered_df[filtered_df['_merge'] == 'left_only'].drop(columns='_merge')

In [71]:
filtered_df

,sentence,news_link,outlet,topic,type,group_id,num_sent,Label_bias,Label_opinion,article,biased_words4
0,YouTube is making clear there will be no “birt...,https://eu.usatoday.com/story/tech/2020/02/03/...,usa-today,elections-2020,center,1.0,1.0,Biased,Somewhat factual but also opinionated,YouTube says no ‘deepfakes’ or ‘birther’ video...,"['belated', 'birtherism']"
1,The increasingly bitter dispute between Americ...,https://www.nbcnews.com/news/sports/women-s-te...,msnbc,sport,left,1.0,1.0,Non-biased,Entirely factual,"FRISCO, Texas — The increasingly bitter disput...",['bitter']
2,So while there may be a humanitarian crisis dr...,https://www.alternet.org/2019/01/here-are-5-of...,alternet,immigration,left,1.0,1.0,Biased,Expresses writer’s opinion,Speaking to the country for the first time fro...,['crisis']
3,A professor who teaches climate change classes...,https://www.breitbart.com/politics/2019/05/09/...,breitbart,environment,right,1.0,1.0,Non-biased,No agreement,A professor who teaches climate change classes...,['legitimate']
4,"Looking around the United States, there is nev...",https://thefederalist.com/2020/03/11/woman-who...,federalist,abortion,right,1.0,1.0,Biased,Somewhat factual but also opinionated,The left has a thing for taking babies hostage...,"['killing', 'never', 'developing', 'humans', '..."
...,...,...,...,...,...,...,...,...,...,...,...
1692,In every case legislators are being swarmed by...,https://www.alternet.org/2019/04/republicans-w...,alternet,gender,left,85.0,1.0,Biased,Expresses writer’s opinion,There are many reasons that Republicans and co...,"['deceit', 'hysteria', 'swarmed', 'right-wing']"
1693,Polls show the transgender ideology is deeply ...,https://www.breitbart.com/politics/2019/02/21/...,breitbart,gender,right,85.0,1.0,Biased,Somewhat factual but also opinionated,A man’s penis becomes a female penis once a ma...,"['ideology', 'unpopular']"
1694,Democrats and Republicans stood and applauded ...,https://eu.usatoday.com/story/news/politics/20...,usa-today,gender,center,85.0,1.0,Non-biased,Entirely factual,(WASHINGTON – New Mexico Rep. Debra Haaland go...,['saluted']
1695,"As a self-described Democratic socialist, Sen....",https://www.foxnews.com/politics/how-did-berni...,fox-news,middle-class,right,85.0,1.0,Non-biased,Entirely factual,"As a self-described Democratic socialist, Sen....",['outspoken']


In [72]:
#filtered_df['article'] = filtered_df['article'].fillna(filtered_df['full_article'])

In [73]:
filtered_df.dropna(subset = ['article'], inplace = True)

In [74]:
filtered_df.shape

(1690, 11)

In [75]:
filtered_df.to_csv("../data/full_articles_dataset.csv", index = False)